In [79]:
import json
from collections import defaultdict
import pandas as pd

In [80]:
df = pd.read_csv('candidate-transfer.csv')
df['from'], df['to'], df['hash'] = df['from'].apply(lambda x: x[2:]), df['to'].apply(lambda x: x[2:]), df['hash'].apply(lambda x: x[2:])
df['block_time'] = pd.to_datetime(df['block_time'])
df = df.loc[df['block_time'] > pd.to_datetime('2021-01-01 15:47:18+00:00')]

print("Candidate transactions: ", df.shape[0])
candidate_address = set(df['to'].unique())
print("Candidate addresses: ", len(candidate_address))

Candidate transactions:  89285
Candidate addresses:  4777


In [81]:
# Filter the transactions whose "from" and "to" both interactived with Hop USDC Bridge
df = df[df['from'].isin(candidate_address)]
df = df.sort_values('block_time').drop_duplicates(('from', 'to'), keep='last')
df['from'].value_counts()

781cdc81cc8a315433bdc1c581a649c4c0d61ee7    32
a822390152cb85db84613fde7676e0b36ebfb334    19
c67f6fbd90806efcac2b7afbafb038afe4df5a0c    10
9ab119f2017358dd33894fef1d20f870e3f70850     9
6649a5d4ae7ac73f2f6b6baf3b722d8dcb281e33     9
                                            ..
318fb4d4de8def84b5b836ee57c0dbf87cf50f7d     1
5f67b369e5155edc4270f1e2615b70dac41e93ca     1
119976a1cfe7c109ef52ce3f74707ab574a526a7     1
ba2dc20329b21370b91e1306b1c0b4c0ff9adbd5     1
9633d229b711dcbf06cdd8739238765e0f3135b0     1
Name: from, Length: 726, dtype: int64

In [99]:
for i in df['from'].value_counts()[df['from'].value_counts()>2].index:
    print('0x' + i)

0x781cdc81cc8a315433bdc1c581a649c4c0d61ee7
0xa822390152cb85db84613fde7676e0b36ebfb334
0xc67f6fbd90806efcac2b7afbafb038afe4df5a0c
0x9ab119f2017358dd33894fef1d20f870e3f70850
0x6649a5d4ae7ac73f2f6b6baf3b722d8dcb281e33
0x1cb15998a75f4628a2bdcbd86a632a33d906c116
0x74c3b2d22ed5990b9ab1f77bd3054d4fd4afcf96
0x80a03dbf383c6fdd96fb95d1a24d63c7b6d02b08
0x7da66c591bfc8d99291385b8221c69ab9b3e0f0e
0x0359f194b3b801a1cce464eb8367a5a7b796a69d
0x9520386ba02883e58075a4cf42e8bea4f5a1be88
0x792a97fbe51ac086360021117f403954291ad332
0xd9529704efb175ecdc7683a1e4d4ce80550c4019
0x214ddd9a30d8328549814d84ccdf41fb5d8ec864
0x32ca3698bc7707102a7e0da52a383f1c881d80d7
0x036c1308b25fc70c3e6be413f5158bdf15e2770d
0xad503b72fc36a699bf849bb2ed4c3db1967a73da
0x11eb197cc9f186e7235f91aa14d0ae20df139a4a
0x090f923e1484912285d07f7e643599b8ebf945ce
0x26c1e466ce41070585d6876dc17b7bd9295c41d5
0xa86271026a587c4beb528f46452f5c8375e2e95b
0x9d9db1c6d79ee49c855081db0a273bcb303c0d02
0x72bddbfdd873dc896db057a09586a2f3e19f4169
0x4626751b1

In [94]:
save_file = defaultdict(lambda: [0, []])

for _, (from_address, to_address, _, tx_hash) in df.iterrows():
    save_file[from_address][0] += 1
    save_file[from_address][1].append((to_address, tx_hash))
    
save_file = dict(sorted(list(save_file.items()), key=lambda x: x[1][0], reverse=True))
json.dump(save_file, open('report_address.json', 'w'), indent=2)